In [ ]:
import pathlib

import xarray as xr


paths = list(
    pathlib.Path("/p/project1/training2330/a6/data/ecmwf_era5/nc").rglob(
        "**/*_12.nc"
    )
)

print(paths)

for path in paths:
    ds = xr.open_dataset(path)
    ds.attrs["mean"] = tuple(
        float(ds.sel(level=level)[variable].mean())
        for level in ds.level
        for variable in ds.sel(level=level)
    )
    ds.attrs["mean_std_order"] = tuple(
        f"level={int(level.values)},variable={variable}"
        for level in ds.level
        for variable in ds.sel(level=level)
    )
    ds.attrs["std"] = tuple(
        float(ds.sel(level=level)[variable].std())
        for level in ds.level
        for variable in ds.sel(level=level)
    )
    ds.attrs[
        "mean_description"
    ] = "Contains the global mean for each level and variable, e.g. for levels 300 and 500hPa with variables z and r, it contains (in order): mean(z) at 300hPa, mean(r) at 300hPa, mean(z) at 500hPa, mean(r) at 500hPa."
    ds.attrs[
        "std_description"
    ] = "Contains the global standard deviation for each level and variable, e.g. for levels 300 and 500hPa with variables z and r, it contains (in order): std(z) at 300hPa, std(r) at 300hPa, std(z) at 500hPa, std(r) at 500hPa."
    ds.to_netcdf(f"{path.as_posix().replace('.nc', '')}_with_mean_and_std.nc")
    print(f"{path} processed")